In [86]:
import open3d as o3d
import numpy as np
from numba import njit
import MinkowskiEngine as ME
import torch
import pandas as pd
from glob import glob
import os

In [131]:
@njit
def to_spherical(points, dataset_name):
    spherical_points = []
    for point in points:
        if (np.abs(point[:3]) < 1e-4).all():
            continue

        r = np.linalg.norm(point[:3])

        # Theta is calculated as an angle measured from the y-axis towards the x-axis
        # Shifted to range (0, 360)
        theta = np.arctan2(point[1], point[0]) * 180 / np.pi
        if theta < 0:
            theta += 360

        if dataset_name == "USyd":
            # VLP-16 has 2 deg VRes and (+15, -15 VFoV).
            # Phi calculated from the vertical axis, so (75, 105)
            # Shifted to (0, 30)
            phi = (np.arccos(point[2] / r) * 180 / np.pi) - 75

        elif dataset_name in ['IntensityOxford', 'Oxford']:
            # Oxford scans are built from a 2D scanner.
            # Phi calculated from the vertical axis, so (0, 180)
            phi = np.arccos(point[2] / r) * 180 / np.pi

        elif dataset_name in ['KITTI', 'TUM']:
            # HDL-64 has 0.4 deg VRes and (+2, -24.8 VFoV).
            # Phi calculated from the vertical axis, so (88, 114.8)
            # Shifted to (0, 26.8)
            phi = (np.arccos(point[2] / r) * 180 / np.pi) - 88

        if point.shape[-1] == 4:
            spherical_points.append([r, theta, phi, point[3]])
        else:
            spherical_points.append([r, theta, phi])

    return spherical_points

def load_pc(dataset_folder, output_folder, filename, dataset_name, mink_quantization_size):
    file_path = os.path.join(dataset_folder, filename)

    pc_pnts = np.fromfile(file_path, dtype=np.float64).reshape([-1, 3])
    pc_s_pnts = np.asarray(to_spherical(pc_pnts, dataset_name))
    coords, inds = ME.utils.sparse_quantize(coordinates=torch.tensor(pc_s_pnts, dtype=torch.float),
                                            quantization_size=mink_quantization_size,
                                            return_index=True,
                                            return_inverse=False)
    pc_vox_pnts = pc_pnts[list(inds)]
    pc_diff_pnts = pc_pnts[list(set(range(4096)) - set(inds.tolist()))]
    print(pc_pnts.shape, pc_vox_pnts.shape, pc_diff_pnts.shape)

    pc, pc_s, pc_vox, pc_diff = o3d.geometry.PointCloud(), o3d.geometry.PointCloud(), o3d.geometry.PointCloud(), o3d.geometry.PointCloud()
    pc.points = o3d.utility.Vector3dVector(pc_pnts)   # Original point cloud
    pc_s.points = o3d.utility.Vector3dVector(pc_s_pnts) # Spherical point cloud
    pc_vox.points = o3d.utility.Vector3dVector(pc_vox_pnts) # Voxelized point cloud
    pc_diff.points = o3d.utility.Vector3dVector(pc_diff_pnts) # Differenced point cloud

    filestem = filename.split('.')[0]
    o3d.io.write_point_cloud(os.path.join(output_folder, f'{filestem}.pcd'), pc, write_ascii=True)
    o3d.io.write_point_cloud(os.path.join(output_folder, f'{filestem}_s.pcd'), pc_s, write_ascii=True)
    o3d.io.write_point_cloud(os.path.join(output_folder, f'{filestem}_vox.pcd'), pc_vox, write_ascii=True)
    o3d.io.write_point_cloud(os.path.join(output_folder, f'{filestem}_diff.pcd'), pc_diff, write_ascii=True)

In [132]:
dataset_folder = '/home/xiayan/testdir/datasets/frame_5m'
output_folder = '/home/xiayan/testdir/datasets/tum_viz'
filename = '2016-00011.bin'
file_path = os.path.join(dataset_folder, filename)

mink_quantization_size = [2.5,2.0,1.875]

In [134]:
filenames = glob('/home/xiayan/testdir/datasets/frame_5m/*.bin')

for filename in filenames[:30]:
    load_pc(dataset_folder=dataset_folder,
            output_folder=output_folder,
            filename=filename.split('/')[-1],
            dataset_name='TUM',
            mink_quantization_size=mink_quantization_size)

(4096, 3) (1719, 3) (2377, 3)
(4096, 3) (1782, 3) (2314, 3)
(4096, 3) (1781, 3) (2315, 3)
(4096, 3) (1332, 3) (2764, 3)
(4096, 3) (1411, 3) (2685, 3)
(4096, 3) (1784, 3) (2312, 3)
(4096, 3) (1492, 3) (2604, 3)
(4096, 3) (1340, 3) (2756, 3)
(4096, 3) (1843, 3) (2253, 3)
(4096, 3) (1647, 3) (2449, 3)
(4096, 3) (1693, 3) (2403, 3)
(4096, 3) (1534, 3) (2562, 3)
(4096, 3) (1407, 3) (2689, 3)
(4096, 3) (1297, 3) (2799, 3)
(4096, 3) (1954, 3) (2142, 3)
(4096, 3) (1297, 3) (2799, 3)
(4096, 3) (2023, 3) (2073, 3)
(4096, 3) (1466, 3) (2630, 3)
(4096, 3) (1586, 3) (2510, 3)
(4096, 3) (1951, 3) (2145, 3)
(4096, 3) (1765, 3) (2331, 3)
(4096, 3) (1852, 3) (2244, 3)
(4096, 3) (1597, 3) (2499, 3)
(4096, 3) (1753, 3) (2343, 3)
(4096, 3) (1510, 3) (2586, 3)
(4096, 3) (1395, 3) (2701, 3)
(4096, 3) (1905, 3) (2191, 3)
(4096, 3) (1564, 3) (2532, 3)
(4096, 3) (2263, 3) (1833, 3)
(4096, 3) (1815, 3) (2281, 3)


In [81]:
pc_pnts

array([[-0.05950609, -0.23934637, -0.04017662],
       [ 0.05591073,  0.01711923,  0.051661  ],
       [-0.11016081, -0.08922945, -0.04351812],
       ...,
       [-0.12106692, -0.11145936, -0.03723965],
       [ 0.04860582, -0.00350828,  0.02706704],
       [-0.02477835, -0.17228568, -0.0329213 ]])

In [82]:
pc_vox_pnts

array([[-0.05950609, -0.23934637, -0.04017662],
       [ 0.05591073,  0.01711923,  0.051661  ],
       [-0.11016081, -0.08922945, -0.04351812],
       ...,
       [ 0.01023923, -0.08604984, -0.00269689],
       [ 0.01180906, -0.08887551, -0.00244572],
       [-0.06026954, -0.24093684, -0.03165236]])

In [83]:
# ! scp -r /home/xiayan/testdir/datasets/tum_viz username@host:path

UnboundLocalError: local variable 'child' referenced before assignment